# 线性回归的简洁实现

通过使用深度学习的框架来简洁地实现
线性回归模型
生成数据集

In [71]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [72]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
print(data_iter)


# 假设 data_iter 是一个迭代器对象
'''
for batch_features, batch_labels in data_iter:
    print("Batch Features:")
    print(batch_features)
    print("Batch Labels:")
    print(batch_labels)
    break  # 仅打印第一个批次的数据
'''
next(iter(data_iter))

[tensor([[-0.4570,  0.3668],
         [ 0.9301,  0.0657],
         [ 0.2169,  0.0454],
         [-0.4854, -0.7284],
         [-2.5032, -1.0657],
         [-1.5230, -1.2886],
         [ 1.5051,  0.7477],
         [-0.6498, -1.1087],
         [ 0.2381,  0.4956],
         [ 0.9326,  0.0248]]),
 tensor([[2.0292],
         [5.8411],
         [4.4802],
         [5.6978],
         [2.8180],
         [5.5425],
         [4.6669],
         [6.6625],
         [2.9962],
         [5.9746]])]

使用框架的预定义好的层

In [73]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))#Sequential是一个存放神经网络层的容器

初始化模型参数
normal(0,0.01)方差为0，均值为0.01

In [74]:
net[0].weight.data.normal_(0, 0.01)#第0层中权重的数据替换为正态分布
net[0].bias.data.fill_(0)#偏差层的用权重用0替代

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [75]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [76]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
#torch.optim.SGD属于torch中优化器用来调整模型的参数，lr学习率

训练过程代码与我们从零开始实现时所做的非常相似

In [94]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        
        l = loss(net(X) ,y)
        # 将trainer.zero_grad()放在l.backward()之前，
        # 可以确保在反向传播之前清零参数的梯度，
        # 以实现独立的梯度计算。
        trainer.zero_grad()
        l.backward()
        
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000109
epoch 2, loss 0.000109
epoch 3, loss 0.000109


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [78]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0005, -0.0010])
b的估计误差： tensor([-0.0002])
